In [14]:
import pandas as pd

In [16]:
# Load the uploaded files into dataframes
odmg_df = pd.read_csv('Data/Revised/Migration/ODMG.csv')
imd_df = pd.read_csv('Data/Revised/Migration/IMD_2021.csv')

# Merge the two dataframes based on 'Migrant MSOA one year ago code' from ODMG Data and 'MSOA21CD' from IMD 2021 Data
merged_df = odmg_df.merge(imd_df[['MSOA21CD', 'Decile']], 
                          left_on='Code_2020', 
                          right_on='MSOA21CD', 
                          how='left')

# Rename the Decile column to 'Before'
merged_df.rename(columns={'Decile': 'IMD_Before'}, inplace=True)

# Drop the 'MSOA21CD' column from the merged dataframe
merged_df = merged_df.drop(columns=['MSOA21CD'])

# Merge the Decile based on 'Middle layer Super Output Areas code' with 'MSOA21CD' from the second dataframe
merged_df = merged_df.merge(imd_df[['MSOA21CD', 'Decile']],
                            left_on='Code_2021',
                            right_on='MSOA21CD',
                            how='left')

# Rename the Decile column to 'After'
merged_df.rename(columns={'Decile': 'IMD_After'}, inplace=True)

# Drop the 'MSOA21CD' column used for merging
merged_df = merged_df.drop(columns=['MSOA21CD'])

# Calculate the difference between 'Before' and 'After' and create a new column 'IMD_Change'
merged_df['IMD_Change'] = merged_df['IMD_After'] - merged_df['IMD_Before']

# Move the 'Count' column to the last position and 'IMD_Change' to be the second last
cols = merged_df.columns.tolist()
cols.append(cols.pop(cols.index('Count')))
cols.insert(-1, cols.pop(cols.index('IMD_Change')))
merged_df = merged_df[cols]

# Export the updated dataframe to a new CSV file
output_path_updated = 'Data/Revised/Migration/Migration.csv'
merged_df.to_csv(output_path_updated, index=False)